In [5]:
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython import display 
import numpy as np
from wrapper import AtariWrapper

noop_max = 0
frame_skip = 0
screen_size = 84
terminal_on_life_loss = False
clip_reward = False
action_repeat_probability = 0.0
normalize = True
# init env and model struct
env = gym.make('BreakoutNoFrameskip-v4', render_mode="human")
# env = AtariWrapper(env, noop_max = noop_max, frame_skip = frame_skip, screen_size = screen_size, terminal_on_life_loss = terminal_on_life_loss, clip_reward = clip_reward, action_repeat_probability = action_repeat_probability, normalize = normalize)



observation = env.reset()
test_episode = 1
total_reward = []
reward_ = 0
for _ in range(test_episode):
    done = False
    truncated = False
    while not(done or truncated):
        action = env.action_space.sample() # 采取一个动作
        observation, reward, done, truncated, info = env.step(action) # 更新状态信息
        reward_ += reward
        env.render()
        if done or truncated:
            observation, info = env.reset()
            total_reward.append(reward_)
            reward_ = 0
#             print(info["lives"], reward_)
print(np.mean(reward_))

env.close()

0.0


In [20]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from collections import deque
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

import matplotlib.pyplot as plt 


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

model = tf.keras.models.load_model("./breakout_20240314_215651/model_100")

noop_max = 0
frame_skip = 4
screen_size = 84
terminal_on_life_loss = False
clip_reward = False
action_repeat_probability = 0.0
normalize = True

# init env and model struct
env = gym.make("BreakoutNoFrameskip-v4", render_mode="human")
env = AtariWrapper(env, noop_max = noop_max, frame_skip = frame_skip, screen_size = screen_size, terminal_on_life_loss = terminal_on_life_loss, clip_reward = clip_reward, action_repeat_probability = action_repeat_probability, normalize = normalize)


state = tf.convert_to_tensor(env.reset()[0])
state = tf.stack([state, state, state, state], axis=-1)
state = tf.expand_dims(state, 0)

total_reward = []
reward_ = 0
for _ in range(10):
    done = False
    truncated = False
    while not (done or truncated):
        traj_info = model(state)
        action = traj_info['a'].numpy()
        next_state, reward, done, truncated, info = env.step(action[0])  # 更新状态信息
        reward_ += reward

        next_state = tf.expand_dims(next_state, 0)
        next_state = tf.stack([next_state, state[:, :, :, 0], state[:, :, :, 1], state[:, :, :, 2]], axis=-1)
        state = next_state
        
        env.render()
#         print(reward)
        
        if done or truncated:
            state = tf.convert_to_tensor(env.reset()[0])
            state = tf.stack([state, state, state, state], axis=-1)
            state = tf.expand_dims(state, 0)
            total_reward.append(reward_)
            reward_ = 0
            break

average_reward = np.mean(total_reward)
        
        
print(f"mean reward = {average_reward}")
env.close()

D:\Anaconda\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


KeyboardInterrupt: 

In [25]:
def random_sample(inds, minibatch_size):
    num_full_batches = inds // minibatch_size

    for i in range(num_full_batches):
        batch_inds = np.arange(i * minibatch_size, (i + 1) * minibatch_size)
        yield tf.convert_to_tensor(batch_inds, dtype=tf.int64)

    r = inds % minibatch_size
    if r:
        batch_inds = np.arange(inds - r, inds)
        yield tf.convert_to_tensor(batch_inds, dtype=tf.int64)

In [36]:
def random_batch_seq_sample(inds, minibatch_size):
    # 按顺序生成所有可能的批次
    all_batches = [np.arange(i * minibatch_size, min((i + 1) * minibatch_size, inds)) for i in range((inds + minibatch_size - 1) // minibatch_size)]
    
    # 随机排列这些批次的索引
    perm_inds = np.random.permutation(len(all_batches))
    
    for ind in perm_inds:
        yield tf.convert_to_tensor(all_batches[ind], dtype=tf.int64)  


for idxes in random_batch_seq_sample(2048, 32):

    print(idxes)

tf.Tensor(
[800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817
 818 819 820 821 822 823 824 825 826 827 828 829 830 831], shape=(32,), dtype=int64)
tf.Tensor(
[864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881
 882 883 884 885 886 887 888 889 890 891 892 893 894 895], shape=(32,), dtype=int64)
tf.Tensor(
[1728 1729 1730 1731 1732 1733 1734 1735 1736 1737 1738 1739 1740 1741
 1742 1743 1744 1745 1746 1747 1748 1749 1750 1751 1752 1753 1754 1755
 1756 1757 1758 1759], shape=(32,), dtype=int64)
tf.Tensor(
[2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029
 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043
 2044 2045 2046 2047], shape=(32,), dtype=int64)
tf.Tensor(
[448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465
 466 467 468 469 470 471 472 473 474 475 476 477 478 479], shape=(32,), dtype=int64)
tf.Tensor(
[192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 